In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np

import efd_utils
import df_utils

%matplotlib inline

In [ ]:
DATE_FORMAT = "%Y-%m-%d"

In [ ]:
# Note this is in local time!!!!!
start_str = "2019-10-20"
end_str = "2019-10-21"

In [ ]:
start = datetime.strptime(start_str, DATE_FORMAT)
end = datetime.strptime(end_str, DATE_FORMAT)

In [ ]:
location = "tucson"

In [ ]:
client = efd_utils.get_client(location)

In [ ]:
csc = efd_utils.CSC("ATHeaderService", 0)

In [ ]:
query = efd_utils.get_base_query(columns=["private_sndStamp",
                                          "heartbeat"],
                                 csc_name=csc.name,
                                 csc_index=csc.index,
                                 topic_name="logevent_heartbeat")
query += " WHERE " + efd_utils.get_time_clause(date_range=(start, end))
#print(query)

In [ ]:
hb_df = await client.query(query)

In [ ]:
ok_to_plot = True
expected_messages = int((end - start).total_seconds())
try:
    times_s = hb_df.private_sndStamp.values
    if times_s.shape[0] != expected_messages:
        print(f"Total Number of Messages: {times_s.shape[0]}, Expected: {expected_messages}")
    diff_s = (times_s[1:] - times_s[:-1] - 1) * 1000
    mean_diff_s = np.mean(diff_s)
    median_diff_s = np.median(diff_s)
    print(f"Mean: {mean_diff_s:.2f} ms")
    print(f"Median: {median_diff_s:.2f} ms")
except AttributeError:
    print("NO DATA FOUND!!!")
    ok_to_plot = False

In [ ]:
if ok_to_plot:
    time_fmt = "%Y-%m-%d %H:%M"
    xlabel = '$\Delta$T (ms)'
    fig = plt.figure(figsize=(8, 3))
    fig.suptitle(f"{end.strftime(time_fmt)} - {start.strftime(time_fmt)}")
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.hist(diff_s, bins=50)
    ax1.set_xlabel(xlabel)
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.hist(diff_s, bins=50, log=True)
    ax2.set_xlabel(xlabel)
    mean_color = 'red'
    median_color = 'orange'
    ax2.axvline(mean_diff_s, color=mean_color)
    ax2.axvline(median_diff_s, color=median_color)
    ax2.annotate(f"Mean: {mean_diff_s:.2f} ms", (0.5, 0.85), xycoords="axes fraction", color=mean_color)
    ax2.annotate(f"Median: {median_diff_s:.2f} ms", (0.5, 0.75), xycoords="axes fraction", color=median_color)